# NN (fully connected feed-forward network)


In [35]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Get the Data

In [36]:
df = pd.read_csv("/content/ds_salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary              3755 non-null   int64 
 5   salary_currency     3755 non-null   object
 6   salary_in_usd       3755 non-null   int64 
 7   employee_residence  3755 non-null   object
 8   remote_ratio        3755 non-null   int64 
 9   company_location    3755 non-null   object
 10  company_size        3755 non-null   object
dtypes: int64(4), object(7)
memory usage: 322.8+ KB


# Data Preprocessing

In [38]:
df.isnull().sum()

,0
work_year,0
experience_level,0
employment_type,0
job_title,0
salary,0
salary_currency,0
salary_in_usd,0
employee_residence,0
remote_ratio,0
company_location,0


In [39]:
print(df['work_year'].value_counts())

work_year
2023    1785
2022    1664
2021     230
2020      76
Name: count, dtype: int64


In [40]:
# Drop the 'salary' and 'salary_currency' columns directly from the original dataset
df.drop(columns=['salary', 'salary_currency'], inplace=True)

In [41]:
df.head()

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,CA,100,CA,M


In [42]:
# Цільова змінна
target = 'experience_level'

# Видалення цільової змінної з фіч
X = df.drop(columns=[target])

# Цільова змінна
y = df[target]

In [43]:
# Кодування цільової змінної
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# One-hot encoding для інших категоріальних змінних
X = pd.get_dummies(X, columns=['employment_type', 'job_title',
                               'employee_residence', 'company_location', 'company_size'])

In [44]:
# Масштабування числових фіч
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [50]:
from keras.utils import to_categorical

# Розділення на навчальну та тестову вибірки
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, to_categorical(y_encoded), test_size=0.2, random_state=42)

# Verification

In [51]:
print("Розмірності X (фічі):", X.shape)

Розмірності X (фічі): (3755, 253)


In [52]:
print("Розмірності y (цільова змінна):", y.shape)

Розмірності y (цільова змінна): (3755,)


In [53]:
print("Кількість класів у цільовій змінній:", len(np.unique(y_encoded)))

Кількість класів у цільовій змінній: 4


In [54]:
print("Типи даних у X:\n", X.dtypes)

Типи даних у X:
 work_year              int64
salary_in_usd          int64
remote_ratio           int64
employment_type_CT      bool
employment_type_FL      bool
                       ...  
company_location_US     bool
company_location_VN     bool
company_size_L          bool
company_size_M          bool
company_size_S          bool
Length: 253, dtype: object


# Create Model

In [64]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input

# Створення моделі
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Визначення вхідного розміру
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(label_encoder.classes_), activation='softmax')  # Вихідний шар з кількістю класів
])

# Компіляція моделі
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [65]:
# Навчання моделі
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, callbacks=[early_stopping])

Epoch 1/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5852 - loss: 1.1728 - val_accuracy: 0.7004 - val_loss: 0.8421
Epoch 2/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7195 - loss: 0.8295 - val_accuracy: 0.7111 - val_loss: 0.7910
Epoch 3/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7526 - loss: 0.7117 - val_accuracy: 0.7057 - val_loss: 0.7817
Epoch 4/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7631 - loss: 0.6807 - val_accuracy: 0.7071 - val_loss: 0.7768
Epoch 5/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7415 - loss: 0.6889 - val_accuracy: 0.7124 - val_loss: 0.7798
Epoch 6/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7651 - loss: 0.6399 - val_accuracy: 0.7204 - val_loss: 0.7719
Epoch 7/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7667 - loss: 0.6442 - val_accuracy: 0.7190 - val_loss: 0.7773
Epoch 8/50
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7791 - loss: 0.6376 - val_accuracy: 0.7164 - val_loss:

In [66]:
# Оцінка на тестовій вибірці
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Точність моделі: {accuracy * 100:.2f}%")

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7306 - loss: 0.7645 
Точність моделі: 72.04%


# kNN, Decision Tree, SVM, Random Forest, AdaBoost

In [68]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# Цільова змінна
y = df['experience_level']
X = pd.get_dummies(df.drop(columns=['experience_level']),
                   columns=['employment_type', 'job_title', 'employee_residence', 'company_location', 'company_size'])

# Кодування цільової змінної
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Масштабування числових фіч
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Розділення даних
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3755 entries, 0 to 3754
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           3755 non-null   int64 
 1   experience_level    3755 non-null   object
 2   employment_type     3755 non-null   object
 3   job_title           3755 non-null   object
 4   salary_in_usd       3755 non-null   int64 
 5   employee_residence  3755 non-null   object
 6   remote_ratio        3755 non-null   int64 
 7   company_location    3755 non-null   object
 8   company_size        3755 non-null   object
dtypes: int64(3), object(6)
memory usage: 264.1+ KB


In [70]:
df.head()

,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,CA,100,CA,M


In [71]:
# Навчання та оцінка моделей
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import accuracy_score

# Список моделей
models = {
    'kNN': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(kernel='rbf', random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=100, random_state=42)
}

# Навчання і оцінка
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy

# Виведення результатів
for model_name, accuracy in results.items():
    print(f"{model_name}: Точність = {accuracy * 100:.2f}%")

kNN: Точність = 70.57%
Decision Tree: Точність = 68.58%
SVM: Точність = 71.24%
Random Forest: Точність = 71.50%
AdaBoost: Точність = 67.11%


#Висновок: нейронна мережа справилась з задачею краще, ніж інші алгоритми. Точність NN = 72,04 %.